# 모듈 Import

In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

# 데이터 Read 함수

In [2]:
def read(what, how):
#원하는 데이터 연도별로 다 불러오가
#what은 무슨 기록인지 받는 입력   예)팀타자, 개인투수....
#how는 생성할 데이터 프레임 이름  예)df_raw, df_bat...
    year = [16,17,18,19,20]
    for i in year:
        globals()['{0}{1}'.format(how,i)] = pd.read_csv("원본데이터/2020빅콘테스트_스포츠투아이_제공데이터_{0}_20{1}.csv".format(what,i), encoding='cp949')

# 팀별 최근경기 전적

In [3]:
read('팀타자', 'hit_')
read('팀투수', 'pit_')

In [4]:
pit_data = pd.concat([pit_16, pit_17, pit_18, pit_19, pit_20])
hit_data = pd.concat([hit_16, hit_17, hit_18, hit_19, hit_20])

In [5]:
def team_sep(df,team):
    df_new =df[df['T_ID']==team]
    return df_new

In [6]:
team = list(pit_16['T_ID'][:10])
year = [16, 17, 18, 19, 20]

In [7]:
# 각 팀별, 시즌별로 데이터 형성. ex) LG_16 == LG 16시즌 데이터
for i in year:
    for j in team:
        globals()['{0}_{1}'.format(j,i)] = team_sep(globals()['pit_{}'.format(i)],j)

In [8]:
def near_play(data):
    temp_data = list(data['WLS'])  # 데이터의 'WLS' column을 불러옴.
    WLD = []
    for i in range(len(temp_data)):
        temp = 0
        if i == 0: # 처음에는 0 대입
            WLD.append(temp)
            continue
        g_cnt = 0
        j = i - 1
        while True:
            if g_cnt == 10:  # 게임 수가 10이 되면 종료
                break
            elif j < 0:     # 인덱스의 위치가 0이 넘어가면 종료
                break
            else:
                check = temp_data[j]   # 인덱스의 위치의 값을 확인하여서 W이면 0.1을 더하고
                if check == 'W':      # L이면 0, D이면 0.5를 더해서 최근 10경기의 전적을 수치화
                    temp += 0.1
                elif check == 'D':
                    temp += 0.05
                elif check == 'L':
                    pass
            j -= 1
            g_cnt += 1
        WLD.append(temp)
    return WLD

In [9]:
for i in year:
    for j in team:
        globals()['{0}_{1}'.format(j,i)]['WLD'] = near_play(globals()['{0}_{1}'.format(j,i)])

In [10]:
# 각 팀별 데이터를 시즌별로 합침
data_16 = pd.concat([NC_16, OB_16, WO_16, HT_16, LG_16, SS_16, KT_16, LT_16, SK_16, HH_16])
data_17 = pd.concat([NC_17, OB_17, WO_17, HT_17, LG_17, SS_17, KT_17, LT_17, SK_17, HH_17])
data_18 = pd.concat([NC_18, OB_18, WO_18, HT_18, LG_18, SS_18, KT_18, LT_18, SK_18, HH_18])
data_19 = pd.concat([NC_19, OB_19, WO_19, HT_19, LG_19, SS_19, KT_19, LT_19, SK_19, HH_19])
data_20 = pd.concat([NC_20, OB_20, WO_20, HT_20, LG_20, SS_20, KT_20, LT_20, SK_20, HH_20])

In [11]:
# 합친 데이터를 정렬함.
data_16 = data_16.sort_index()
data_17 = data_17.sort_index()
data_18 = data_18.sort_index()
data_19 = data_19.sort_index()
data_20 = data_20.sort_index()

In [12]:
data_last = data_16
for i in year:
    if i == 16:
        continue
    data_last = pd.concat([data_last, globals()['data_{0}'.format(i)]])

In [13]:
data_last = data_last.loc[:, ['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'WLD']]
data_last = data_last.reset_index()
del data_last['index']
data_last = data_last.sort_values(by=['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID'], ascending=True)

In [14]:
data_last

,G_ID,GDAY_DS,T_ID,VS_T_ID,WLD
1,20160401HHLG0,20160401,HH,LG,0.0
0,20160401HHLG0,20160401,LG,HH,0.0
3,20160401HTNC0,20160401,HT,NC,0.0
2,20160401HTNC0,20160401,NC,HT,0.0
5,20160401KTSK0,20160401,KT,SK,0.0
...,...,...,...,...,...
6394,20200719LTSS0,20200719,SS,LT,0.5
6396,20200719OBHT0,20200719,HT,OB,0.6
6397,20200719OBHT0,20200719,OB,HT,0.6
6398,20200719WOSK0,20200719,SK,WO,0.4


# Park Factor

In [15]:
read("경기", 'game_')

In [16]:
game_data = pd.concat([game_16, game_17, game_18, game_19, game_20])
names = ['STADIUM', 'PARK']

# 각 구장의 파크팩터는 http://old.statiz.co.kr/stadium.php?opt=1&st=all에서 가져옴.
park_data = pd.read_csv("원본데이터/STATIZ_파크팩터.csv", encoding='cp949', names = names)

pls_ = list(park_data['STADIUM'])

park_ = []
for i in game_data['STADIUM']:
    std_list = list(park_data['STADIUM'])
    cur = std_list.index(i)
    temp = park_data['PARK'][cur]
    park_.append(temp)
    
game_data['PARK'] = park_
game_data = game_data.reset_index()
game_data = game_data.drop(['index'], axis=1)

del game_data['HEADER_NO']
del game_data['GWEEK']
del game_data['STADIUM']
game_data

,G_ID,GDAY_DS,VISIT_KEY,HOME_KEY,PARK
0,20160401HHLG0,20160401,HH,LG,938
1,20160401HTNC0,20160401,HT,NC,964
2,20160401KTSK0,20160401,KT,SK,1019
3,20160401LTWO0,20160401,LT,WO,983
4,20160401OBSS0,20160401,OB,SS,1067
...,...,...,...,...,...
3195,20200719HHLG0,20200719,HH,LG,938
3196,20200719KTNC0,20200719,KT,NC,1038
3197,20200719LTSS0,20200719,LT,SS,1067
3198,20200719OBHT0,20200719,OB,HT,1013


In [17]:
df = game_data
names = df.columns

# 기본 데이터 틀과 맞추기 위해서 각 열을 하니씩 더 늘리는 작업을 진행하였습니다.
for i in range(len(df['G_ID'].values)):
    temp = df.loc[i:i]
    if i==0:
        df2 = pd.concat([temp, temp], ignore_index=True)
    else:
        df2 = pd.concat([df2, temp], ignore_index=True)
        df2 = pd.concat([df2, temp], ignore_index=True)

df2.rename(columns = {'HOME_KEY' : 'T_ID', 'VISIT_KEY':'VS_T_ID'}, inplace = True)        
df2 = df2[['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'PARK']]

for idx in range(len(df2)):
    if idx%2==0:
        df2['T_ID'][idx], df2['VS_T_ID'][idx] = df2['VS_T_ID'][idx], df2['T_ID'][idx]

df2 = df2.sort_values(by=['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID'], ascending=True)
df2

,G_ID,GDAY_DS,T_ID,VS_T_ID,PARK
0,20160401HHLG0,20160401,HH,LG,938
1,20160401HHLG0,20160401,LG,HH,938
2,20160401HTNC0,20160401,HT,NC,964
3,20160401HTNC0,20160401,NC,HT,964
4,20160401KTSK0,20160401,KT,SK,1019
...,...,...,...,...,...
6395,20200719LTSS0,20200719,SS,LT,1067
6397,20200719OBHT0,20200719,HT,OB,1013
6396,20200719OBHT0,20200719,OB,HT,1013
6399,20200719WOSK0,20200719,SK,WO,1019


In [18]:
df_total = df2
df_total['WLD'] = data_last['WLD']

In [19]:
df_total

,G_ID,GDAY_DS,T_ID,VS_T_ID,PARK,WLD
0,20160401HHLG0,20160401,HH,LG,938,0.0
1,20160401HHLG0,20160401,LG,HH,938,0.0
2,20160401HTNC0,20160401,HT,NC,964,0.0
3,20160401HTNC0,20160401,NC,HT,964,0.0
4,20160401KTSK0,20160401,KT,SK,1019,0.0
...,...,...,...,...,...,...
6395,20200719LTSS0,20200719,SS,LT,1067,0.5
6397,20200719OBHT0,20200719,HT,OB,1013,0.6
6396,20200719OBHT0,20200719,OB,HT,1013,0.6
6399,20200719WOSK0,20200719,SK,WO,1019,0.5


# BPF & PPF

In [20]:
read('경기', 'home')
home_data = pd.concat([home16, home17, home18, home19, home20], ignore_index=True)
gday = {'LG':{'home':[], 'away':[]}, 'NC':{'home':[], 'away':[]}, 'SK':{'home':[], 'away':[]},
        'WO':{'home':[], 'away':[]}, 'SS':{'home':[], 'away':[]}, 'HT':{'home':[], 'away':[]},
        'OB':{'home':[], 'away':[]}, 'LT':{'home':[], 'away':[]}, 'KT':{'home':[], 'away':[]},
        'HH':{'home':[], 'away':[]}}

# 각 팀의 홈 경기 일자와 원정 경기 일자를 나눠서 딕셔너리에 넣어준다.
def set_gday(data):
    for i in range(len(data['G_ID'])):
        home = data['HOME_KEY'][i]
        away = data['VISIT_KEY'][i]
        gid = data['G_ID'][i]
        gday[home]['home'].append(gid)
        gday[away]['away'].append(gid)
        
set_gday(home_data)

In [21]:
read('팀투수', 'pit_')
pit_data = pd.concat([pit_16, pit_17, pit_18, pit_19, pit_20], ignore_index=True)

teams = pit_data['T_ID'].loc[0:9].values
park_df = pd.DataFrame(index=teams)

# 홈 경기에서의 승리 수와 원정경기에서의 패배 수를 계산
game_temp = {}
for team in teams:
    arr_game = []
    w_cnt = 0
    l_cnt = 0
    
    for game in gday[team]:
        g_cnt = len(gday[team][game])
        arr_game.append(g_cnt)
        
        if game == 'home':
            for day in gday[team][game]:
                temp_df = pit_data[ pit_data['G_ID']==day ].reset_index()
                wls_temp = temp_df['WLS'][0]
                if wls_temp == 'W':
                    w_cnt += 1
            arr_game.append(w_cnt)
        elif game == 'away':
            for day in gday[team][game]:
                temp_df = pit_data[ pit_data['G_ID']==day ].reset_index()
                wls_temp = temp_df['WLS'][1]
                if wls_temp == 'L':
                    l_cnt += 1
            arr_game.append(l_cnt)
    game_temp[team] = arr_game

# 홈경기의 승률과 원정 경기의 패률을 계산, 이를 토대로 각 팀의 IPC 계산
for team in game_temp:
    W_rate = game_temp[team][1] / game_temp[team][0]
    L_rate = game_temp[team][3] / game_temp[team][2]
    IPC = (18.5 - W_rate) / (18.5 - L_rate)
    game_temp[team].append(IPC)
    
game_temp

{'LG': [323, 175, 317, 168, 0.9993418369163186],
 'HH': [318, 149, 323, 191, 1.0068557753832486],
 'NC': [320, 178, 319, 157, 0.9964411611106277],
 'HT': [319, 185, 319, 178, 0.9987769721324365],
 'SK': [321, 176, 320, 163, 0.9978371177009486],
 'KT': [320, 145, 320, 196, 1.008909853249476],
 'WO': [321, 188, 321, 163, 0.9956713704441174],
 'LT': [317, 166, 321, 191, 1.0039852740802133],
 'SS': [324, 152, 317, 181, 1.0056802939416352],
 'OB': [317, 200, 323, 126, 0.9867021986564679]}

In [22]:
read('팀타자', 'hit_')
hit_data = pd.concat([hit_16, hit_17, hit_18, hit_19, hit_20], ignore_index=True)

# 홈 득점, 홈 실점, 원정 득점, 원정 실점 계산
for team in teams:
    
    for game in gday[team]:
        
        get_score = 0
        lose_score = 0
        g_list = gday[team][game]
        
        for day in g_list:
            hit_temp = hit_data[ hit_data['G_ID'] == day ].reset_index()
            pit_temp = pit_data[ pit_data['G_ID'] == day ].reset_index()
            if game == 'home':
                get_score += hit_temp['RUN'][0]
                lose_score += pit_temp['R'][0]
            elif game == 'away':
                get_score += hit_temp['RUN'][1]
                lose_score += pit_temp['R'][1]
        
        game_temp[team].append(get_score)
        game_temp[team].append(lose_score)
        
game_temp

{'LG': [323, 175, 317, 168, 0.9993418369163186, 1529, 1537, 1701, 1724],
 'HH': [318, 149, 323, 191, 1.0068557753832486, 1524, 1836, 1649, 1776],
 'NC': [320, 178, 319, 157, 0.9964411611106277, 1690, 1590, 1708, 1615],
 'HT': [319, 185, 319, 178, 0.9987769721324365, 1799, 1607, 1799, 1760],
 'SK': [321, 176, 320, 163, 0.9978371177009486, 1619, 1610, 1546, 1557],
 'KT': [320, 145, 320, 196, 1.008909853249476, 1555, 1850, 1535, 1805],
 'WO': [321, 188, 321, 163, 0.9956713704441174, 1711, 1547, 1769, 1662],
 'LT': [317, 166, 321, 191, 1.0039852740802133, 1686, 1751, 1638, 1746],
 'SS': [324, 152, 317, 181, 1.0056802939416352, 1854, 1933, 1559, 1695],
 'OB': [317, 200, 323, 126, 0.9867021986564679, 1680, 1413, 2068, 1605]}

In [23]:
# init factor를 계산.
for team in game_temp:
    h_get = game_temp[team][-4]
    h_lose = game_temp[team][-3]
    a_get = game_temp[team][-2]
    a_lose = game_temp[team][-1]
    
    h_game = game_temp[team][0]
    a_game = game_temp[team][2]
    
    init_fac = ((h_get + h_lose) / h_game) / ((a_get + a_lose) / a_game)
    
    game_temp[team].append(init_fac)

In [24]:
park_df = pd.DataFrame(game_temp)
park_df = park_df.transpose()

columns = ['H_game', 'Home_Win', 'A_game', 'Away_Lose', 'IPC', 'Home_R', 'Home_RA', 'Away_R', 'Away_RA', 'Init_Fac']
park_df.columns = columns

# team rate 계산
team_rate = park_df['Init_Fac'] / park_df['IPC']
park_df['Team_Rate'] = team_rate.values

# OPC, SF, SF1, RHT, RAT, OHT, OAT 계산
OPC = 10 / (10 - 1 + team_rate)
park_df['OPC'] = OPC.values
SF = OPC * team_rate
SF1 = 1 - ((SF - 1) / (10 - 1))

RHT = park_df['Home_R'] / park_df['H_game']
RAT = park_df['Away_R'] / park_df['A_game']
OHT = park_df['Home_RA'] / park_df['H_game']
OAT = park_df['Away_RA'] / park_df['A_game']

# RAL 계산
tot = 0
for i in hit_data['RUN']:
    tot += i
RAL = tot / len(hit_data['RUN'])

# TBR과 TPR을 계산. 이후 어느정도 안정되는 지점을 찾아 거기의 값을 사용하기로 결정.
TBR = 1
TPR = 1
check_list = []
for i in range(100):
    TBR = abs(RAT / SF1 + RHT / SF) * abs(1 + ((TPR - 1) / (10 - 1))) / RAL
    TPR = abs(OAT / SF1 + OHT / SF) * abs(1 + ((TBR - 1) / (10 - 1))) / RAL
    check_list.append((TBR.values, TPR.values))
TBR, TPR = check_list[5]

# BPF, PPF 계산 
BPF = (SF + SF1) / abs(2 * (1 + (TPR - 1) / 9))
PPF = (SF + SF1) / abs(2 * (1 + (TBR - 1) / 9))

In [25]:
park_df['BPF'] = BPF
park_df['PPF'] = PPF
park_final = park_df.loc[:, ['BPF', 'PPF']]
park_final

,BPF,PPF
LG,0.827749,0.829172
HH,0.858404,0.879156
NC,0.878132,0.868933
HT,0.851834,0.841000
SK,0.907633,0.907751
KT,0.866062,0.892822
WO,0.862672,0.849878
LT,0.877477,0.885673
SS,0.917426,0.927864
OB,0.831685,0.798610


### pre는 19년도 이전 post는 19, 20년도

In [26]:
# NC는 19년도 부터 홈 구장이 바뀌었으므로 두 가지로 나눠서 bpf, ppf 계산 해야 함.
pre19 = pd.concat([home16, home17, home18], ignore_index=True)
post19 = pd.concat([home19, home20], ignore_index=True)

In [27]:
pre_pit = pd.concat([pit_16, pit_17, pit_18])
post_pit = pd.concat([pit_19, pit_20])

pre_hit = pd.concat([hit_16, hit_17, hit_18])
post_hit = pd.concat([hit_19, hit_20])

In [28]:
# 19년도 이전과 이후의 홈 경기와 원정 경기일자 입력.
NC_pre = {'home':[], 'away':[]}
for i in range(len(pre19['G_ID'])):
    gid = pre19['G_ID'][i]
    if pre19['HOME_KEY'][i] == 'NC':
        NC_pre['home'].append(gid)
    elif pre19['VISIT_KEY'][i] == 'NC':
        NC_pre['away'].append(gid)
        
NC_post = {'home':[], 'away':[]}
for i in range(len(post19['G_ID'])):
    gid = post19['G_ID'][i]
    if post19['HOME_KEY'][i] == 'NC':
        NC_post['home'].append(gid)
    elif post19['VISIT_KEY'][i] == 'NC':
        NC_post['away'].append(gid)

In [29]:
NC_temp = {'NC_pre':[], 'NC_post':[]}

# 19년도 이전과 이후의 홈 경기수, 홈 승수, 원정 경기수, 원정 패수를 계산
for team in NC_temp:
    arr_game = []
    w_cnt = 0
    l_cnt = 0
    
    if team == 'NC_pre':
        for game in NC_pre:
            g_cnt = len(NC_pre[game])
            arr_game.append(g_cnt)

            if game == 'home':
                for day in NC_pre[game]:
                    temp_df = pre_pit[ pre_pit['G_ID']==day ].reset_index()
                    wls_temp = temp_df['WLS'][0]
                    if wls_temp == 'W':
                        w_cnt += 1
                arr_game.append(w_cnt)
            elif game == 'away':
                for day in NC_pre[game]:
                    temp_df = pre_pit[ pre_pit['G_ID']==day ].reset_index()
                    wls_temp = temp_df['WLS'][1]
                    if wls_temp == 'L':
                        l_cnt += 1
                arr_game.append(l_cnt)
    
    elif team == 'NC_post':
        for game in NC_post:
            g_cnt = len(NC_post[game])
            arr_game.append(g_cnt)

            if game == 'home':
                for day in NC_post[game]:
                    temp_df = post_pit[ post_pit['G_ID']==day ].reset_index()
                    wls_temp = temp_df['WLS'][0]
                    if wls_temp == 'W':
                        w_cnt += 1
                arr_game.append(w_cnt)
            elif game == 'away':
                for day in NC_post[game]:
                    temp_df = post_pit[ post_pit['G_ID']==day ].reset_index()
                    wls_temp = temp_df['WLS'][1]
                    if wls_temp == 'L':
                        l_cnt += 1
                arr_game.append(l_cnt)
    
    NC_temp[team] = arr_game

In [30]:
NC_df = pd.DataFrame(NC_temp)
NC_df = NC_df.transpose()
NC_df.columns = ['Game_H', 'Win_H', 'Game_A', 'Lose_A']

In [31]:
# 홈 승수와 원정 패수를 토대로 승률과 패률을 계산하고, IPC 계산
W_rate = NC_df['Win_H'] / NC_df['Game_H']
L_rate = NC_df['Lose_A'] / NC_df['Game_A']

IPC = (18.5 - W_rate) / (18.5 - L_rate)
NC_df['IPC'] = IPC

In [32]:
# 19년도 이전과 이후의 홈과 원정에서의 득점과 실점 계산
# 해당 게임키의 데이터들을 보면 첫번째 열이 홈팀이고, 두번째 열이 원정팀이다.
game_arr = []
for team in NC_temp:
    temp_arr = []
    if team == 'NC_pre':
        for game in NC_pre:

            get_score = 0
            lose_score = 0
            g_list = NC_pre[game]

            for day in g_list:
                hit_temp = pre_hit[ pre_hit['G_ID'] == day ].reset_index()
                pit_temp = pre_pit[ pre_pit['G_ID'] == day ].reset_index()
                if game == 'home':
                    get_score += hit_temp['RUN'][0]
                    lose_score += pit_temp['R'][0]
                elif game == 'away':
                    get_score += hit_temp['RUN'][1]
                    lose_score += pit_temp['R'][1]
            temp_arr.append(get_score)
            temp_arr.append(lose_score)
            
    elif team == 'NC_post':
        for game in NC_post:

            get_score = 0
            lose_score = 0
            g_list = NC_post[game]

            for day in g_list:
                hit_temp = post_hit[ post_hit['G_ID'] == day ].reset_index()
                pit_temp = post_pit[ post_pit['G_ID'] == day ].reset_index()
                if game == 'home':
                    get_score += hit_temp['RUN'][0]
                    lose_score += pit_temp['R'][0]
                elif game == 'away':
                    get_score += hit_temp['RUN'][1]
                    lose_score += pit_temp['R'][1]
            temp_arr.append(get_score)
            temp_arr.append(lose_score)
    
    game_arr.append(temp_arr)

In [33]:
NC_pre_arr = game_arr[0]
NC_post_arr = game_arr[1]

NC_df = NC_df.loc[:, ['Game_H', 'Win_H', 'Game_A', 'Lose_A']]

i = 0
for column in ['R_H', 'RA_H', 'R_A', 'RA_A']:
    NC_df[column] = [NC_pre_arr[i], NC_post_arr[i]]
    i += 1
    
NC_df['IPC'] = IPC

In [34]:
# init factor, team rate, OPC, SF, SF1, RHT, RAT, OHT, OAT, RAL을 계산
init_fac = ((NC_df['R_H'] + NC_df['RA_H']) / NC_df['Game_H']) / ((NC_df['R_A'] + NC_df['RA_A']) / NC_df['Game_A'])
team_rate = init_fac / NC_df['IPC']
OPC = 10 / (10 - 1 + team_rate)
SF = OPC * team_rate
SF1 = 1 - ((SF - 1) / (10 - 1))
RHT = NC_df['R_H'] / NC_df['Game_H']
RAT = NC_df['R_A'] / NC_df['Game_A']
OHT = NC_df['RA_H'] / NC_df['Game_H']
OAT = NC_df['RA_A'] / NC_df['Game_A']

In [35]:
RAL = []
tot = 0
for i in pre_hit['RUN']:
    tot += i
RAL.append( tot / len(pre_hit['RUN']) )
tot = 0
for i in post_hit['RUN']:
    tot += i  
RAL.append( tot / len(post_hit['RUN']) )

In [36]:
NC_df['RAL'] = RAL

In [37]:
# TBR, TPR계산
TBR = 1
TPR = 1
RAL = NC_df['RAL']
check_list = []
for i in range(100):
    TBR = abs(RAT / SF1 + RHT / SF) * abs(1 + ((TPR - 1) / (10 - 1))) / RAL
    TPR = abs(OAT / SF1 + OHT / SF) * abs(1 + ((TBR - 1) / (10 - 1))) / RAL
    check_list.append((TBR.values, TPR.values))

In [38]:
TBR, TPR = check_list[6]

In [39]:
# BPF, PPF 계산
BPF = (SF + SF1) / abs(2 * (1 + (TPR - 1) / 9))
PPF = (SF + SF1) / abs(2 * (1 + (TBR - 1) / 9))

NC_df['BPF'] = BPF
NC_df['PPF'] = PPF

In [40]:
park_final = park_final.drop('NC')

In [41]:
NC_df

,Game_H,Win_H,Game_A,Lose_A,R_H,RA_H,R_A,RA_A,IPC,RAL,BPF,PPF
NC_pre,216,116,216,108,1144,1110,1159,1169,0.997942,5.497685,0.871238,0.869567
NC_post,104,62,103,49,546,480,549,446,0.993319,4.744712,0.894590,0.867302


### Make Data

In [42]:
df_total['BPF'] = np.nan
df_total['PPF'] = np.nan

df_total

,G_ID,GDAY_DS,T_ID,VS_T_ID,PARK,WLD,BPF,PPF
0,20160401HHLG0,20160401,HH,LG,938,0.0,NaN,NaN
1,20160401HHLG0,20160401,LG,HH,938,0.0,NaN,NaN
2,20160401HTNC0,20160401,HT,NC,964,0.0,NaN,NaN
3,20160401HTNC0,20160401,NC,HT,964,0.0,NaN,NaN
4,20160401KTSK0,20160401,KT,SK,1019,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...
6395,20200719LTSS0,20200719,SS,LT,1067,0.5,NaN,NaN
6397,20200719OBHT0,20200719,HT,OB,1013,0.6,NaN,NaN
6396,20200719OBHT0,20200719,OB,HT,1013,0.6,NaN,NaN
6399,20200719WOSK0,20200719,SK,WO,1019,0.5,NaN,NaN


In [43]:
# 각 구장의 BPF와 PPF 대입.
for i in range(len(df_total['T_ID'])):
    home = df_total['T_ID'][i]
    temp = df_total['G_ID'][i][:4]
    if int(temp) <= 2019 and home == 'NC':
        df_total['BPF'][i] = NC_df['BPF']['NC_pre']
        df_total['PPF'][i] = NC_df['PPF']['NC_pre']
        
    elif int(temp) >= 2019 and home == 'NC':
        df_total['BPF'][i] = NC_df['BPF']['NC_pre']
        df_total['PPF'][i] = NC_df['PPF']['NC_pre']
    
    else:
        df_total['BPF'][i] = park_final['BPF'][home]
        df_total['PPF'][i] = park_final['PPF'][home]

In [44]:
df_total

,G_ID,GDAY_DS,T_ID,VS_T_ID,PARK,WLD,BPF,PPF
0,20160401HHLG0,20160401,HH,LG,938,0.0,0.858404,0.879156
1,20160401HHLG0,20160401,LG,HH,938,0.0,0.827749,0.829172
2,20160401HTNC0,20160401,HT,NC,964,0.0,0.851834,0.841000
3,20160401HTNC0,20160401,NC,HT,964,0.0,0.871238,0.869567
4,20160401KTSK0,20160401,KT,SK,1019,0.0,0.866062,0.892822
...,...,...,...,...,...,...,...,...
6395,20200719LTSS0,20200719,SS,LT,1067,0.5,0.917426,0.927864
6397,20200719OBHT0,20200719,HT,OB,1013,0.6,0.851834,0.841000
6396,20200719OBHT0,20200719,OB,HT,1013,0.6,0.831685,0.798610
6399,20200719WOSK0,20200719,SK,WO,1019,0.5,0.907633,0.907751


In [45]:
df_total.to_csv('모델input_경기외데이터.csv', index = False, encoding = 'cp949')